# **Ingeniería de Características - `05_ingenieria_caracteristicas.ipynb`**

### **🎯 Objetivo del Notebook**
Este notebook tiene como objetivo aplicar técnicas de **ingeniería de características** sobre las señales EEG, con el fin de capturar mejor la información contenida en las señales y mejorar el rendimiento de los modelos de clasificación.

### **📌 Contexto**
En el notebook anterior (`04_analisis_caracteristicas.ipynb`), analizamos la importancia de las características existentes y aplicamos varias técnicas de selección y reducción de dimensionalidad. Sin embargo, concluimos que:

- La **eliminación de outliers** fue el único preprocesamiento que mejoró el rendimiento.
- La **selección de características** y **reducción de dimensionalidad** no sólo no aportaron mejoras, sino que en muchos casos **empeoraron los resultados**.

Dado que los datos EEG son señales **temporales complejas**, y que el modelo actual no logra capturar suficientemente su estructura dinámica con las características crudas, es necesario **generar nuevas variables** que representen mejor la información relevante en la señal.

---

### **🧠 ¿Por qué Ingeniería de Características?**

Incluso los mejores modelos no pueden rendir bien si no se les alimenta con datos representativos. Por eso, vamos a construir nuevas variables derivadas de las señales originales que puedan captar:

- Tendencias locales (media móvil)
- Cambios rápidos (derivadas o gradientes)
- Patrones en el dominio de la frecuencia (FFT)
- Niveles de variabilidad o actividad (desviación estándar, rango, energía)

Estas transformaciones buscan **extraer información latente** que no es evidente en las señales brutas.

---

### **🚀 Flujo de Trabajo en este Notebook**
1️⃣ **Carga de datos**    
2️⃣ **Definición de ventanas temporales para extracción de características**    
3️⃣ **Cálculo de estadísticas temporales (media, std, varianza, gradiente)**  
4️⃣ **Aplicación de transformadas en frecuencia (FFT, potencia espectral)**  
5️⃣ **Generación de nuevas variables y construcción de un nuevo conjunto de datos**  
6️⃣ **Evaluación del impacto en el rendimiento del modelo**  
7️⃣ **Conclusión sobre qué variables se mantienen y próximos pasos**

*Cargamos los datos sin preprocesar y eliminamos `outliers` pero todavía no normalizamos ya que las transformaciones que haremos, como calcular medias, desviaciones, FFT o energía, dependen de la forma y magnitud original de la señal. Si normalizáramos antes, perderíamos esa información significativa. Por eso, aplicaremos la normalización después de extraer las nuevas características.

---

📌 Si estas nuevas variables aportan mejoras, las incorporaremos como parte del preprocesamiento final antes de probar modelos más avanzados como **XGBoost, LightGBM o Redes Neuronales**.


## **1. Carga de datos y Eliminación Outliers**

In [2]:
import os
import pickle
import numpy as np
import pandas as pd
from scipy.stats import zscore


DATA_PATH = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\data\raw_data\train\train"
SUBJECT = "subj1"
SERIES_TRAIN = [f"{SUBJECT}_series{i}_data.csv" for i in range(1, 9)]
SERIES_EVENTS = [f"{SUBJECT}_series{i}_events.csv" for i in range(1, 9)]

def load_data(series, path=DATA_PATH):
    dfs = [pd.read_csv(os.path.join(path, file)) for file in series]
    return pd.concat(dfs, ignore_index=True)

df_train = load_data(SERIES_TRAIN)
df_events = load_data(SERIES_EVENTS)
df = df_train.merge(df_events, on="id")

eeg_columns = df.columns[1:33]  # Columnas de señales EEG
event_columns = ["HandStart", "FirstDigitTouch", "BothStartLoadPhase", "LiftOff", "Replace", "BothReleased"]

df_sujeto1 = df[df["id"].str.startswith("subj1_series")]
df_train = df_sujeto1[df_sujeto1["id"].str.contains("series[1-6]_")]
df_valid = df_sujeto1[df_sujeto1["id"].str.contains(r"series[7-8]_", regex=True)]

X_train, y_train = df_train[eeg_columns], df_train[event_columns]
X_valid, y_valid = df_valid[eeg_columns], df_valid[event_columns]

# Eliminación de Outliers
z_scores = np.abs(zscore(X_train))
outlier_threshold = 3
mask = (z_scores < outlier_threshold).all(axis=1)
X_train_filtered = X_train[mask]
y_train_filtered = y_train[mask]

print(f"X_train_filtered shape: {X_train_filtered.shape}")
print(f"y_train_filtered shape: {y_train_filtered.shape}")
print(f"X_valid shape: {X_valid.shape}")
print(f"y_valid shape: {y_valid.shape}")

X_train_filtered shape: (1043205, 32)
y_train_filtered shape: (1043205, 6)
X_valid shape: (236894, 32)
y_valid shape: (236894, 6)


## **2. Creación Ventanas Temporales**

Las ventanas sirven para dividir la señal EEG continua en segmentos cortos que nos permiten extraer estadísticas (como media, energía o frecuencia) en intervalos de tiempo. Así captamos cómo cambia la actividad cerebral antes, durante y después de un evento.

In [3]:
# Parámetros
window_size = 128  # duración de la ventana
step_size = 64     # paso entre ventanas

def create_windows(X, y, window_size, step_size):
    X_windows, y_windows = [], []
    for i in range(0, X.shape[0] - window_size, step_size):
        window = X.iloc[i:i+window_size]
        label = y.iloc[i + window_size//2]  # tomamos la etiqueta del centro
        X_windows.append(window)
        y_windows.append(label)
    return np.array(X_windows), pd.DataFrame(y_windows, columns=y.columns)

X_train_win, y_train_win = create_windows(X_train_filtered, y_train_filtered, window_size, step_size)
X_valid_win, y_valid_win = create_windows(X_valid, y_valid, window_size, step_size)

print("Ventanas creadas:")
print(f"X_train_win shape: {X_train_win.shape}")
print(f"y_train_win shape: {y_train_win.shape}")

Ventanas creadas:
X_train_win shape: (16299, 128, 32)
y_train_win shape: (16299, 6)


## **3. Extraer Estadísticas Temporales**

**Para cada ventana y cada canal EEG vamos a sacar:**
- Media
- Desviación estándar
- Mínimo y máximo
- Rango (max - min)
- Mediana
- Percentiles 25 y 75
- Gradiente medio
- Asimetría (skewness)
- Curtosis

In [4]:
def extract_time_features(X_windows):
    features = []
    for window in X_windows:
        stats = []
        for ch in window.T:  # recorremos canales
            ch_series = pd.Series(ch)
            stats.extend([
                ch_series.mean(),
                ch_series.std(),
                ch_series.min(),
                ch_series.max(),
                ch_series.max() - ch_series.min(),  # rango
                ch_series.median(),
                ch_series.quantile(0.25),
                ch_series.quantile(0.75),
                np.gradient(ch).mean(),
                ch_series.skew(),
                ch_series.kurt()
            ])
        features.append(stats)
    return np.array(features)

X_train_feats = extract_time_features(X_train_win)
X_valid_feats = extract_time_features(X_valid_win)

print("Nuevas características extraídas:")
print(f"X_train_feats shape: {X_train_feats.shape}")

Nuevas características extraídas:
X_train_feats shape: (16299, 352)


### **Versión optimizada del extractor temporal:**

Dado que el proceso de extracción de características temporales tardaba demasiado, optimizamos el código utilizando funciones más rápidas y vectorizadas, como `numpy` y `scipy.stats` en lugar de `pandas`. Esto permitió reducir significativamente el tiempo de cómputo sin perder precisión en los resultados.

In [7]:
from scipy.stats import skew, kurtosis

def extract_time_features_fast(X_windows):
    features = []
    for window in X_windows:
        stats = []
        for ch in window.T:
            stats.extend([
                np.mean(ch),
                np.std(ch),
                np.min(ch),
                np.max(ch),
                np.ptp(ch),  # rango: max - min
                np.median(ch),
                np.percentile(ch, 25),
                np.percentile(ch, 75),
                np.mean(np.gradient(ch)),
                skew(ch),           # scipy (más rápido)
                kurtosis(ch)        # scipy (más rápido)
            ])
        features.append(stats)
    return np.array(features)

X_train_feats = extract_time_features_fast(X_train_win)
X_valid_feats = extract_time_features_fast(X_valid_win)

print("Nuevas características extraídas:")
print(f"X_train_feats shape: {X_train_feats.shape}")

Nuevas características extraídas:
X_train_feats shape: (16299, 352)


## **4. Extraer Características en Frecuencia (FFT)**

El cerebro se comunica en diferentes frecuencias (ondas delta, theta, alfa, beta...), la FFT ayuda a ver qué tan activas están esas bandas durante cada ventana. Al trabajar con señales EEG, es fundamental capturar información tanto en el dominio temporal como en el de frecuencia. Para ello, en lugar de usar una FFT directa, utilizamos el método de Welch (`scipy.signal.welch`), que estima la potencia espectral de forma más estable y robusta frente al ruido. Esto nos permite calcular de forma más fiable la energía presente en las bandas características del EEG (delta, theta, alfa y beta), generando variables más representativas para los modelos de clasificación.

In [10]:
from scipy.signal import welch

def extract_freq_features(X_windows, fs=128):
    freq_feats = []
    for window in X_windows:
        features = []
        for ch in window.T:
            freqs, psd = welch(ch, fs=fs, nperseg=128)
            bands = {
                'delta': (0.5, 4),
                'theta': (4, 8),
                'alpha': (8, 13),
                'beta': (13, 30)
            }
            for low, high in bands.values():
                band_power = np.sum(psd[(freqs >= low) & (freqs < high)])
                features.append(band_power)
            features.append(np.sum(psd))  # energía total
        freq_feats.append(features)
    return np.array(freq_feats)

X_train_freq = extract_freq_features(X_train_win)
X_valid_freq = extract_freq_features(X_valid_win)

print("Características en frecuencia extraídas:")
print(f"X_train_freq shape: {X_train_freq.shape}")

Características en frecuencia extraídas:
X_train_freq shape: (16299, 160)


## **5. Generación Nuevas Variables**

Unimos los datos de las estadísticas temporales y de las características en frecuencia, y normalizamos los datos.

In [11]:
X_train_final = np.concatenate([X_train_feats, X_train_freq], axis=1)
X_valid_final = np.concatenate([X_valid_feats, X_valid_freq], axis=1)

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_final)
X_valid_scaled = scaler.transform(X_valid_final)

## **6. Entrenar y Evaluar Modelos**

Finalmente entrenamos con estas nuevas variables el modelo base que hasta ahora mejores resultados ha demostrado, **Regresión Logística**, y lo evaluamos para verificar si estas nuevas características realmente mejoran el rendimiento.

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def evaluate_model(X_train, y_train, X_valid, y_valid, event_name):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)
    print(f"{event_name}: AUC = {auc:.4f}")
    return auc

# Evaluación por evento
for i, event in enumerate(y_train_win.columns):
    print(f"\nEvaluando evento: {event}")
    auc = evaluate_model(
        X_train_scaled, y_train_win[event],
        X_valid_scaled, y_valid_win[event],
        event
    )


Evaluando evento: HandStart
HandStart: AUC = 0.9539

Evaluando evento: FirstDigitTouch
FirstDigitTouch: AUC = 0.9040

Evaluando evento: BothStartLoadPhase
BothStartLoadPhase: AUC = 0.9404

Evaluando evento: LiftOff
LiftOff: AUC = 0.8833

Evaluando evento: Replace
Replace: AUC = 0.8892

Evaluando evento: BothReleased
BothReleased: AUC = 0.8587


In [14]:
import os
import pickle

# Ruta de guardado
processed_path = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\data\processed"
os.makedirs(processed_path, exist_ok=True)

# Guardar datos preprocesados finales (w/o outliers + features temporales + frecuencia + escalado)
with open(os.path.join(processed_path, "preprocessed_features_temporal_freq.pkl"), "wb") as f:
    pickle.dump((X_train_scaled, y_train_win, X_valid_scaled, y_valid_win), f)

# Guardar en CSV para visualización rápida si hace falta
pd.DataFrame(X_train_scaled).to_csv(os.path.join(processed_path, "X_train_feats.csv"), index=False)
pd.DataFrame(y_train_win).to_csv(os.path.join(processed_path, "y_train_feats.csv"), index=False)
pd.DataFrame(X_valid_scaled).to_csv(os.path.join(processed_path, "X_valid_feats.csv"), index=False)
pd.DataFrame(y_valid_win).to_csv(os.path.join(processed_path, "y_valid_feats.csv"), index=False)

# Guardar resultados AUC del modelo actual
auc_dict = {
    "HandStart": 0.9539,
    "FirstDigitTouch": 0.9040,
    "BothStartLoadPhase": 0.9404,
    "LiftOff": 0.8833,
    "Replace": 0.8892,
    "BothReleased": 0.8587
}

auc_df = pd.DataFrame.from_dict(auc_dict, orient='index', columns=['AUC'])
auc_df.index.name = 'Evento'
auc_df.to_csv(os.path.join(processed_path, "auc_results_feats_logreg.csv"))

## 📌 **Conclusión y Próximos Pasos**
### Comparación de Rendimiento: Antes vs Después de Ingeniería de Características

Comprobamos si las nuevas variables generadas (estadísticas temporales + frecuencia mediante potencia espectral) mejoran el rendimiento del modelo. A continuación, se comparan los valores de AUC-ROC obtenidos con:

- **Datos preprocesados únicamente eliminando outliers y normalizando**
- **Nuevas características extraídas y normalizadas**

| Evento                | AUC (antes) | AUC (con nuevas características) |
|-----------------------|-------------|----------------------------------|
| HandStart             | 0.718       | 0.9539                           |
| FirstDigitTouch       | 0.694       | 0.9040                           |
| BothStartLoadPhase    | 0.6922      | 0.9404                           |
| LiftOff               | 0.7462      | 0.8833                           |
| Replace               | 0.8501      | 0.8892                           |
| BothReleased          | 0.808       | 0.8587                           |

🟢 **Conclusión**: Las nuevas características extraídas mejoran de forma clara y consistente el rendimiento del modelo en todos los eventos. Se observa un aumento especialmente significativo en eventos como *HandStart* y *BothStartLoadPhase*, donde se alcanzan valores superiores a 0.94 en AUC-ROC. Por tanto, estas nuevas variables se incorporarán al pipeline final de modelado. Tras mostrarse los resultados, se han guardado los datos preprocesados para ser utilizados en el siguiente notebook con modelos avanzados.

### ✅ Preprocesado Completo

El conjunto de datos final preparado para entrenar modelos incluye los siguientes pasos:

- **Filtrado de outliers**: Eliminación de muestras extremas en el conjunto de entrenamiento mediante z-score (no se eliminan en validación).
- **División en ventanas temporales**: Cada ventana representa 1 segundo de EEG (128 muestras) con un solapamiento del 50%.
- **Extracción de características**: Por cada ventana y canal se calcularon:
  - Estadísticas temporales: media, desviación estándar, percentiles, rango, gradiente, skewness, kurtosis, etc.
  - Características en frecuencia: energía espectral en bandas delta, theta, alpha, beta y energía total.
- **Normalización**: Se aplicó `StandardScaler` sobre las variables extraídas para estandarizarlas (media 0, varianza 1).
- **Alineación correcta con etiquetas**: Las etiquetas (`y_train_win`) se tomaron del centro de cada ventana, manteniendo la correspondencia temporal con los eventos.

---

### Próximos pasos - Notebook **`06_modelado_avanzado`**

En el siguiente notebook entrenaremos modelos más potentes para mejorar el rendimiento,  como:
   - `RandomForestClassifier`
   - `XGBoost`
   - `LightGBM`
   - Redes neuronales con `Keras`